<a href="https://colab.research.google.com/github/rahulcse-code/Rakshak-/blob/main/North_East_Weather_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import random
import webbrowser

# ==========================================
# CONFIGURATION & SETUP
# ==========================================
# We define real coordinates for North East Capitals
NE_CITIES = {
    "1": {"name": "Gangtok, Sikkim", "lat": 27.3314, "lon": 88.6138, "wiki": "Gangtok"},
    "2": {"name": "Shillong, Meghalaya", "lat": 25.5788, "lon": 91.8933, "wiki": "Shillong"},
    "3": {"name": "Guwahati, Assam", "lat": 26.1445, "lon": 91.7362, "wiki": "Guwahati"},
    "4": {"name": "Kohima, Nagaland", "lat": 25.6701, "lon": 94.1077, "wiki": "Kohima"},
    "5": {"name": "Aizawl, Mizoram", "lat": 23.7271, "lon": 92.7176, "wiki": "Aizawl"},
}

# ==========================================
# SOURCE 1: REAL-TIME WEATHER API (Open-Meteo)
# ==========================================
def fetch_live_weather(lat, lon):
    """
    Connects to Open-Meteo Satellite API to get live weather metrics.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": "true",
        "hourly": "temperature_2m,relativehumidity_2m,precipitation"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status() # Raise error if website is down
        data = response.json()

        current = data['current_weather']
        return {
            "temp": current['temperature'],
            "wind": current['windspeed'],
            "code": current['weathercode'] # WMO Weather interpretation codes
        }
    except Exception as e:
        print(f"❌ CRITICAL ERROR: Could not connect to Weather Satellite. {e}")
        exit() # STOP execution. No fake data allowed.

# ==========================================
# SOURCE 2: LIVE WEB SCRAPING (Wikipedia)
# ==========================================
def scrape_city_description(wiki_suffix):
    """
    Scrapes the first paragraph of the city's Wikipedia page.
    """
    url = f"https://en.wikipedia.org/wiki/{wiki_suffix}"

    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Wikipedia content is usually in <p> tags under mw-parser-output
        # We grab the first meaningful paragraph
        paragraphs = soup.find_all('p')
        for p in paragraphs:
            text = p.get_text().strip()
            if len(text) > 50: # Skip empty or short header paragraphs
                return text[:300] + "..." # Return first 300 chars

        return "Description not found on Wikipedia."
    except Exception as e:
        return f"Could not scrape Wikipedia: {e}"

# ==========================================
# COMPONENT 3: MACHINE LEARNING ENGINE
# ==========================================
def train_activity_predictor():
    """
    Trains a Random Forest Classifier on a synthetic expert knowledge base.
    Features: [Temperature, WindSpeed, WeatherCode]
    Target: Activity Class
    """
    # Creating a "Synthetic Historical Dataset" representing 1000 days of expert decisions
    # This simulates having a CSV of past user preferences.
    X_train = []
    y_train = []

    for _ in range(1000):
        # Simulate varied weather conditions
        t = random.uniform(0, 35) # Temp range 0-35C
        w = random.uniform(0, 25) # Wind range
        c = random.choice([0, 1, 2, 3, 51, 61, 80, 95]) # WMO codes (Clear to Storm)

        # EXPERT LOGIC for Ground Truth Labels
        if c >= 80: label = "Stay Indoors" # Storm/Rain
        elif c >= 51: label = "Cafe/Museum" # Drizzle
        elif t > 30: label = "Water Park/Pool" # Hot
        elif t < 10 and w > 10: label = "Cafe/Museum" # Cold & Windy
        elif t < 10: label = "Bonfire/Hot Spring" # Cold but calm
        elif t > 15 and c < 3: label = "Trekking/Nature" # Pleasant
        else: label = "City Walk"

        X_train.append([t, w, c])
        y_train.append(label)

    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    return clf

# ==========================================
# COMPONENT 4: AGENTIC AI (MAIN SYSTEM)
# ==========================================
def run_smart_agent():
    print("\n" + "="*50)
    print("   ECO-SMART NORTH EAST TOURISM AGENT (V2.0)")
    print("   Capabilities: Live Weather API + Wikipedia Scraping + ML Analysis")
    print("="*50)

    # 1. User Selection
    print("\nSelect Area of Interest:")
    for k, v in NE_CITIES.items():
        print(f"  {k}. {v['name']}")

    choice = input("\nEnter Choice (1-5): ")
    city_data = NE_CITIES.get(choice)

    if not city_data:
        print("❌ Invalid Selection.")
        return

    print(f"\n🔄 AGENT STATUS: Connecting to Satellite & Wikipedia for {city_data['name']}...")

    # 2. Parallel Data Collection (API + Scraping)
    weather = fetch_live_weather(city_data['lat'], city_data['lon'])
    description = scrape_city_description(city_data['wiki'])

    print("✅ Data Acquisition Complete.")

    # 3. ML Analysis
    print("🧠 Running Random Forest Classifier...")
    ml_model = train_activity_predictor()

    # Predict based on live data
    prediction = ml_model.predict([[weather['temp'], weather['wind'], weather['code']]])[0]

    # 4. Generate Report
    print("\n" + "-"*50)
    print(f"📄 INTELLIGENT REPORT: {city_data['name']}")
    print("-"*50)
    print(f"🌍 About Location: \"{description}\"")
    print(f"\n🌤️  LIVE WEATHER CONDITIONS:")
    print(f"    • Temperature: {weather['temp']}°C")
    print(f"    • Wind Speed:  {weather['wind']} km/h")
    print(f"    • Condition Code: {weather['code']} (WMO Standard)")
    print("-"*50)
    print(f"🤖 AI RECOMMENDATION: {prediction}")
    print(f"   Reasoning: Based on {weather['temp']}°C and current sky conditions.")
    print("-"*50)

    # 5. Agent Action (Real-world interaction)
    nav_q = input("🚀 Open recommended locations on Google Maps? (y/n): ")
    if nav_q.lower() == 'y':
        query = f"{prediction} in {city_data['name']}"
        url = f"https://www.google.com/maps/search/{query}"
        webbrowser.open(url)
        print(f"✅ Browser opened: {url}")

if __name__ == "__main__":
    run_smart_agent()


   ECO-SMART NORTH EAST TOURISM AGENT (V2.0)
   Capabilities: Live Weather API + Wikipedia Scraping + ML Analysis

Select Area of Interest:
  1. Gangtok, Sikkim
  2. Shillong, Meghalaya
  3. Guwahati, Assam
  4. Kohima, Nagaland
  5. Aizawl, Mizoram

Enter Choice (1-5): 3

🔄 AGENT STATUS: Connecting to Satellite & Wikipedia for Guwahati, Assam...
✅ Data Acquisition Complete.
🧠 Running Random Forest Classifier...

--------------------------------------------------
📄 INTELLIGENT REPORT: Guwahati, Assam
--------------------------------------------------
🌍 About Location: "Description not found on Wikipedia."

🌤️  LIVE WEATHER CONDITIONS:
    • Temperature: 23.8°C
    • Wind Speed:  2.8 km/h
    • Condition Code: 1 (WMO Standard)
--------------------------------------------------
🤖 AI RECOMMENDATION: Trekking/Nature
   Reasoning: Based on 23.8°C and current sky conditions.
--------------------------------------------------
🚀 Open recommended locations on Google Maps? (y/n): y
✅ Browser op